In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = mls
band = [7]
commitnumber = de5d43e
molecule = {'co2': 0.0004}
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band5
  atmpro = mls
  band = 5
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [1, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 1100
  vmin = 980
  w_diffuse = [(1.75,), (1.66, 1.9)]
  wgt = [(0.75,), (0.75, 0.95)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.000854                  NaN       NaN
0.000312      NaN       NaN                  1.0 -0.000871
0.000750      2.0 -0.000857                  2.0 -0.000865
0.001052      3.0 -0.000863                  3.0 -0.000888
0.001476      4.0 -0.000868                  4.0 -0.000898
0.002070      5.0 -0.000874                  5.0 -0.000906
0.002904      6.0 -0.000878                  6.0 -0.000913
0.004074      7.0 -0.000882                  7.0 -0.000919
0.005714      8.0 -0.000886                  8.0 -0.000925
0.008015      9.0 -0.000889                  9.0 -0.000930
0.011243     10.0 -0.000892                 10.0 -0.000937
0.015771     11.0 -0.000893                 11.0 -0.000947
0.022122     12.0 -0.000895                 12.0 -0.000955
0.031031     13.0 -0.000895                 13.0 -0.000962
0.043528     14.0 -0.000895                 14.0 -0.000964
0.061057     15.0 -0.000888                 15.0 -0.000974
0.085645     16.0 -0.000816                 16.0 -0.000976
0.120136     17.0 -0.000526                 17.0 -0.000857
0.168516     18.0  0.000126                 18.0 -0.000497
0.236378     19.0  0.001345                 19.0  0.000238
0.331549     20.0  0.003424                 20.0  0.001535
0.465100     21.0  0.006765                 21.0  0.003677
0.652400     22.0  0.011880                 22.0  0.007250
0.915100     23.0  0.017533                 23.0  0.011314
1.283650     24.0  0.017654                 24.0  0.011564
1.800600     25.0  0.012490                 25.0  0.008216
2.525700     26.0  0.007626                 26.0  0.004961
3.542800     27.0  0.004149                 27.0  0.002550
4.969550     28.0  0.001868                 28.0  0.000920
6.970850     29.0  0.000475                 29.0 -0.000116
9.778100     30.0 -0.000357                 30.0 -0.000762
13.715850    31.0 -0.000860                 31.0 -0.001164
19.239350    32.0 -0.001142                 32.0 -0.001389
26.987250    33.0 -0.001272                 33.0 -0.001495
37.855300    34.0 -0.001339                 34.0 -0.001553
53.100050    35.0 -0.001389                 35.0 -0.001589
73.887500    36.0 -0.001430                 36.0 -0.001612
97.662500    37.0 -0.001481                 37.0 -0.001646
121.437500   38.0 -0.001546                 38.0 -0.001677
145.212500   39.0 -0.001608                 39.0 -0.001665
168.987500   40.0 -0.001685                 40.0 -0.001703
192.762500   41.0 -0.001822                 41.0 -0.001794
216.537500   42.0 -0.001982                 42.0 -0.002002
240.312500   43.0 -0.002094                 43.0 -0.002157
264.087500   44.0 -0.002142                 44.0 -0.002233
287.862500   45.0 -0.002121                 45.0 -0.002233
311.637500   46.0 -0.002026                 46.0 -0.002162
335.412500   47.0 -0.001854                 47.0 -0.001989
359.187500   48.0 -0.001603                 48.0 -0.001750
382.962500   49.0 -0.001271                 49.0 -0.001441
406.737500   50.0 -0.000860                 50.0 -0.001044
430.512500   51.0 -0.000373                 51.0 -0.000566
454.287500   52.0  0.000185                 52.0 -0.000020
478.062500   53.0  0.000807                 53.0  0.000596
501.837500   54.0  0.001492                 54.0  0.001285
525.612500   55.0  0.002239                 55.0  0.002025
549.387500   56.0  0.003052                 56.0  0.002835
573.162500   57.0  0.003931                 57.0  0.003718
596.937500   58.0  0.004877                 58.0  0.004679
620.712500   59.0  0.005891                 59.0  0.005719
644.487500   60.0  0.006973                 60.0  0.006814
668.262500   61.0  0.008122                 61.0  0.007962
692.037500   62.0  0.009338                 62.0  0.009184
715.812500   63.0  0.010622                 63.0  0.010479
739.587500   64.0  0.011972                 64.0  0.011848
76

# Fluxes by Layer

CRD                              CLIRAD            \
                         flug          fldg      fnetg       flug      fldg   
pressure     level                                                            
0.000000e+00 1     -30.877901  0.000000e+00 -30.877901        NaN       NaN   
1.000000e-08 1            NaN           NaN        NaN -30.907964  0.000002   
6.244000e-04 2     -30.877902  2.720626e-08 -30.877902 -30.907964  0.000002   
8.759000e-04 3     -30.877902  3.836629e-08 -30.877902 -30.907964  0.000002   
1.228600e-03 4     -30.877902  5.463926e-08 -30.877902 -30.907964  0.000002   
1.723400e-03 5     -30.877902  7.842983e-08 -30.877902 -30.907964  0.000002   
2.417400e-03 6     -30.877902  1.131986e-07 -30.877902 -30.907964  0.000002   
3.390900e-03 7     -30.877902  1.640090e-07 -30.877902 -30.907965  0.000002   
4.756500e-03 8     -30.877902  2.382544e-07 -30.877902 -30.907965  0.000002   
6.672000e-03 9     -30.877903  3.467223e-07 -30.877902 -30.907965  0.000002   
9.358900e-03 10    -30.877903  5.051716e-07 -30.877902 -30.907966  0.000002   
1.312780e-02 11    -30.877904  7.366003e-07 -30.877903 -30.907966  0.000002   
1.841450e-02 12    -30.877904  1.074581e-06 -30.877903 -30.907967  0.000003   
2.583020e-02 13    -30.877906  1.568109e-06 -30.877904 -30.907968  0.000003   
3.623230e-02 14    -30.877908  2.288701e-06 -30.877905 -30.907970  0.000004   
5.082340e-02 15    -30.877910  3.343441e-06 -30.877907 -30.907972  0.000004   
7.129060e-02 16    -30.877914  5.018858e-06 -30.877909 -30.907976  0.000006   
1.000000e-01 17    -30.877920  8.276457e-06 -30.877912 -30.907982  0.000008   
1.402710e-01 18    -30.877930  1.541378e-05 -30.877914 -30.907991  0.000013   
1.967600e-01 19    -30.877944  3.096564e-05 -30.877913 -30.908006  0.000025   
2.759970e-01 20    -30.877965  6.433707e-05 -30.877901 -30.908027  0.000048   
3.871000e-01 21    -30.877991  1.347824e-04 -30.877856 -30.908057  0.000098   
5.431000e-01 22    -30.878012  2.810998e-04 -30.877731 -30.908092  0.000201   
7.617000e-01 23    -30.878001  5.782859e-04 -30.877423 -30.908123  0.000420   
1.068500e+00 24    -30.877911  1.124923e-03 -30.876786 -30.908125  0.000833   
1.498800e+00 25    -30.877763  1.876851e-03 -30.875886 -30.908122  0.001420   
2.102400e+00 26    -30.877659  2.665974e-03 -30.874993 -30.908188  0.002074   
2.949000e+00 27    -30.877653  3.424257e-03 -30.874228 -30.908367  0.002750   
4.136600e+00 28    -30.877777  4.132661e-03 -30.873645 -30.908689  0.003430   
5.802500e+00 29    -30.878066  4.790437e-03 -30.873276 -30.909188  0.004111   
8.139200e+00 30    -30.878558  5.413971e-03 -30.873144 -30.909906  0.004797   
1.141700e+01 31    -30.879307  6.023876e-03 -30.873283 -30.910898  0.005494   
1.601470e+01 32    -30.880383  6.631451e-03 -30.873752 -30.912235  0.006196   
2.246400e+01 33    -30.881876  7.251242e-03 -30.874624 -30.914003  0.006903   
3.151050e+01 34    -30.883927  7.938605e-03 -30.875988 -30.916355  0.007653   
4.420010e+01 35    -30.886774  8.772277e-03 -30.878001 -30.919550  0.008514   
6.200000e+01 36    -30.890729  9.799473e-03 -30.880930 -30.923921  0.009534   
8.577500e+01 37    -30.895975  1.101818e-02 -30.884956 -30.929651  0.010724   
1.095500e+02 38    -30.901325  1.219639e-02 -30.889129 -30.935426  0.011863   
1.333250e+02 39    -30.906896  1.341192e-02 -30.893484 -30.941305  0.013019   
1.571000e+02 40    -30.912669  1.465751e-02 -30.898012 -30.947141  0.014166   
1.808750e+02 41    -30.918751  1.599388e-02 -30.902757 -30.953150  0.015380   
2.046500e+02 42    -30.925565  1.767717e-02 -30.907887 -30.959684  0.016860   
2.284250e+02 43    -30.933510  2.004213e-02 -30.913468 -30.967413  0.018952   
2.522000e+02 44    -30.942655  2.328915e-02 -30.919366 -30.976371  0.021836   
2.759750e+02 45    -30.952970  2.757046e-02 -30.925400 -30.986468  0.025642   
2.997500e+02 46    -30.964408  3.303504e-02 -30.931373 -30.997619  0.030504   
3.235250e+02 47    -30.976909  3.983071e-02 -30.937078 -31.009784  0.036579 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')